# Likelihood ratio test
To Calculate the significance of the difference between two nested models, we use the likelihood ratio test. The likelihood ratio test is based on the likelihood ratio, which expresses how many times more likely the data are under one model than the other. The likelihood ratio is defined as:
$$
\Lambda = \frac{\mathcal{L}(\theta_1)}{\mathcal{L}(\theta_0)}
$$
where $\mathcal{L}(\theta_1)$ is the likelihood of the data under the alternative hypothesis, and $\mathcal{L}(\theta_0)$ is the likelihood of the data under the null hypothesis. For us, the null hypothesis is the standard model, the background. And the alternative hypothesis our extension to the standard model, new particicles in the extension are the leptoquarks and $Z'$ bosons that contributes with extra events known as the signal, *i.e* the null hipotesis is the SM background and the alternative hipotesis is the SM background plus the signal. The likelihood ratio test is defined as:
$$
\lambda = -2\ln\Lambda = -2\ln\frac{\mathcal{L}(\theta_1)}{\mathcal{L}(\theta_0)}
$$
## Likelihood ratio for binned counts
In each bin the likelihood is defined as a Poisson distribution, then for a histogram with $N_{bins}$ bins, the likelihood is defined as:
$$
\mathcal{L}(\theta) = \prod_{i=1}^{N_{bins}} \frac{\mu_i^{n_i}e^{-\mu_i}}{n_i!}
$$
where $n_i$ is the number of events in the bin $i$, and $\mu_i$ is the expected number of events in the bin $i$ under the hypothesis $\theta$.

For our case, the null hypothesis is the background, and the alternative hypothesis is the background plus the signal. Then, the likelihood ratio in each bin is gived by
$$
\Lambda_i = \frac{\mathcal{L}_i(\theta_1)}{\mathcal{L}_i(\theta_0)} =\frac{\frac{e^{-\left(s_i+b_i\right)}\left(s_i+b_i\right)^{n_i^{\text {cand }}}}{n_i^{\text {cand } !}}}{e^{-b_i} \frac{b_i^{n_i^{\text {cand }}}}{n_i^{\text {cand }} !}}
$$ 
or in a useful form
$$
\lambda_i=-2 \ln \Lambda_i=2 s_i-2 n_i \ln \left(1+\frac{s_i}{b_i}\right)
$$
where $s_i$ is the number of signal events in the bin $i$, and $b_i$ is the number of background events in the bin $i$.

The Likelihood ratio for the whole histogram is the product of the likelihood ratio in each bin, then
$$
\Lambda = \prod_{i=1}^{N_{bins}} \Lambda_i \;\; \Longrightarrow  \;\; \lambda = -2\ln\Lambda = -2\sum_{i=1}^{N_{bins}} \ln \Lambda_i = \sum_{i=1}^{N_{bins}} \lambda_i
$$
Let me define some useful quantities:
- LR weight:
$$
w_i = \ln \left( 1 + \frac{s_i}{b_i} \right)
$$
- Mean of the $\lambda_i$ for "b" expts:
$$
\left< \lambda_i \right>_b = 2s_i -2 b_i w_i
$$
- Mean of the $\lambda_i$ for "s+b" expts:
$$
\left< \lambda_i \right>_{s+b} = 2s_i -2 (s_i+b_i) w_i
$$
- RMS for poisson distribution:
$$
\sigma_{n_i}^2 = n_i
$$
- RMS for $\lambda_i$ for "s+b":
$$
\sigma_{s_i+b_i}^2 = 4(s_i+b_i)w_i^2
$$
- RMS for $\lambda_i$ for "b":
$$
\sigma_{b_i}^2 = 4b_iw_i^2
$$
Significance is defined as the number of standard deviations that the null hypothesis is away from the alternative hypothesis, then
$$
\kappa=\frac{\langle \lambda\rangle_b-N \sigma_b-\langle\lambda\rangle_{s+b}}{\sigma_{s+b}}
$$
where $N$ is the expected number of background sigmas in the critical region.

Simplyfing the expression for the optimized power of analysis:
$$
\kappa=\frac{\sum s_i w_i-N \sqrt{\sum b_i w_i^2}}{\sqrt{\sum\left(s_i+b_i\right) w_i^2}}
$$

For a single bin, and strong signal, the optimized power of analysis is:
$$
\kappa=\frac{s}{\sqrt{s+b}}
$$

With numpy, we can calculate the optimized power for the full histogram with the following code:
  
```python
  def approx_global_sig(sig, bkg, N = 0.0) :
    w = np.log(1. + sig/(bkg + 1e-9))
    s_w = sig * w
    b_w = bkg * w
    s_ww = sig * w ** 2
    b_ww = bkg * w ** 2
    num = np.sum(s_w) - N * np.sqrt(np.sum(b_ww))
    den = np.sqrt(np.sum(s_ww + b_ww))
    return num / den
```

In [4]:
import numpy as np


def approx_global_sig(sig: np.array, bkg: np.array, N: float = 0.0) -> float:
    """
    Calculates the statistical significance of a signal over background in a given
    dataset using a modified version of the formula (S -N sqrt(B))/sqrt(S+B), 
    where S is the number of signal events, B is the number of background events,
    and N is the expected number of background sigmas in the signal region. 

    Parameters:
    sig (np.array): Array containing the number of signal events in each bin.
    bkg (np.array): Array containing the number of background events in each bin.
    N (float): Expected number of background events in the signal region. 
        Default value is 0.0.

    Returns:
    float: The statistical significance of the signal.
    """
    # Calculate weight factor w for each bin
    w = np.log(1. + sig / (bkg + 1e-9))

    # Calculate intermediate quantities
    s_w = sig * w
    b_w = bkg * w
    s_ww = sig * w ** 2
    b_ww = bkg * w ** 2

    # Calculate numerator and denominator of modified formula
    num = np.sum(s_w) - N * np.sqrt(np.sum(b_ww))
    den = np.sqrt(np.sum(s_ww + b_ww))

    # Calculate statistical significance and return it
    return num / den

The number of events for signal and backgrounds are calculated in the following way:
$$
N_{S} = \sigma_{S} \times \mathcal{L} \times \epsilon_{S} \\
$$
where $\sigma_{S}$ is the cross section of the signal process, $\mathcal{L}$ is the integrated luminosity and $\epsilon_{S}$ is the efficiency of the signal process. The efficiency is calculated by dividing the number of events that pass the selection criteria by the total number of events. The number of events for the background processes are calculated in the same way.

In [16]:
import os 
from itertools import product
import pandas as pd
import numpy as np

cases = ["woRHC"]

limits = ['zp_upper_limit', 'zp_lower_limit']

bkgs =  ['tbart', 'V+jets', 'stop', 'Diboson']

channels = {
    'hadronic_dLQ': 'hadronic_Tau_Tau_b_b',
    'hadronic_sLQ': 'hadronic_Tau_Tau_b',
    'hadronic_non-resonant': 'hadronic_Tau_Tau',
    'semileptonic_dLQ': 'semileptonic_Tau_Tau_b_b',
    'semileptonic_sLQ': 'semileptonic_Tau_Tau_b',
    'semileptonic_non-resonant': 'semileptonic_Tau_Tau'
    }

signals = [
    "total_tau_tau",
    ]

M_U= [1000, 1500, 2000, 2500,3000,3500]
G_U = [1.75]
G_ZP = [0.0,0.5, 1.5, 2.5, 3.5]

xs_folder = os.path.join(
    os.path.dirname(os.getcwd()), 
    "01_signal_production", 
    "xs_13TeV"
    )
eff_folder = os.path.join(
    os.path.dirname(os.getcwd()),
     "03_delphes_preselection", "Efficiencies")
significance_folder = os.path.join(os.getcwd(), "Significances")

L = 137 * 1e3 # Integrated luminosity in pb^-1

for case, signal, limit,gu in product(cases, signals,limits,G_U):
    sigificance_path = os.path.join(
        significance_folder,
        case,
        limit,
        f"significances_G_U_{gu}.csv"
        )
    os.makedirs(os.path.dirname(sigificance_path), exist_ok=True)
    xs_path = os.path.join(
        xs_folder,
        case,
        limit,
        signal,
        f"XS_matrix_{gu}.csv"
        )
    xs_matrix = pd.read_csv(xs_path, index_col=0)
    Significance_matrix = pd.DataFrame(index = G_ZP, columns = M_U)

    for m, gzp in product(M_U,G_ZP):
       
        Matrix_Signal = np.zeros([11,6])
        Matrix_BKG = np.zeros([11,6])
        # N = L * xs * eff
        xs = xs_matrix[str(m)][gzp]
        for n, channel in enumerate(channels):

            if m < 2500:
                txt_folder = os.path.join(
                    os.getcwd(), "Data_LQS2023", 
                    f"Histograms_{case}",f"M{m}",
                    channels[channel]
                    )
                scale =1 
            else:
                txt_folder = os.path.join(
                    os.getcwd(), "Data_LQS2023", 
                    f"Histograms_{case}",f"M{2500}",
                    channels[channel]
                )
                scale = xs / xs_matrix["2500"][gzp]

            
            high_per_bin = np.loadtxt(
                os.path.join(txt_folder, f"high_per_bin_tau_tau.txt")
                )
            high_per_bin = high_per_bin*scale
            if gzp != 0.0: 
                eff_path = os.path.join(
                    eff_folder,
                    case,
                    limit,
                    signal,
                    channel,
                    f"eff_matrix_{gu}.csv"
                    )
                eff_matrix = pd.read_csv(eff_path, index_col=0)

               
                eff = eff_matrix[str(m)][gzp]
                N = L * xs * eff
                high_per_bin = high_per_bin*N/sum(high_per_bin)

            
            Matrix_Signal[:, n] += high_per_bin

            for bkg in bkgs:
                Matrix_BKG[:, n] += np.loadtxt(
                    os.path.join(txt_folder, f"high_per_bin_{bkg}.txt")
                    )
        Signal_Data = np.asarray(Matrix_Signal.reshape((1,66)))
        BKG_Data = np.asarray(Matrix_BKG.reshape((1,66)))
        Significance_matrix[m][gzp] = approx_global_sig(Signal_Data, BKG_Data)
    
    Significance_matrix.to_csv(sigificance_path)

'/disco4/personal_folders/c.rodriguez45/github/lq_zprime/05_Significances'